# Scheduled trains maximization with GSA

## 0. Load libraries

In [11]:
%load_ext autoreload
%autoreload 2

import numpy as np

from benchmarks.railway import Solution, RevenueMaximization
from benchmarks.generator import get_schedule_request, get_revenue_behaviour
from src.entities import GSA, Boundaries

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Define corridor

corridor = {"MAD": {
                "CIU": {
                    "COR": {
                        "SEV": {
                            "CAD": {}
                        },
                        "PGE": {
                            "ANT": {
                                "GRA": {},
                                "MAL": {}
                                    }
                                }
                            }
                        }
                    }
            }

In [3]:
np.random.seed(seed=28)

# Dummy schedule
schedule = {1: {'MAD': [0, 0], 'BAR': [148, 152], 'FIG': [180, 180]},
            2: {'MAD': [8, 8], 'ZAR': [28, 30], 'BAR': [165, 167], 'FIG': [210, 210]},
            3: {'MAD': [30, 30], 'BAR': [180, 182], 'FIG': [225, 225]}}

for service in schedule:
    for station in schedule[service]:
        schedule[service][station] = np.array(schedule[service][station], dtype=np.float64)
        
revenue = get_revenue_behaviour(schedule)

schedule, revenue

({1: {'MAD': array([0., 0.]),
   'BAR': array([148., 152.]),
   'FIG': array([180., 180.])},
  2: {'MAD': array([8., 8.]),
   'ZAR': array([28., 30.]),
   'BAR': array([165., 167.]),
   'FIG': array([210., 210.])},
  3: {'MAD': array([30., 30.]),
   'BAR': array([180., 182.]),
   'FIG': array([225., 225.])}},
 {1: {'canon': 222.75,
   'k': 1.05,
   'dt_max_penalty': 31.18,
   'tt_max_penalty': 28.96},
  2: {'canon': 242.0,
   'k': 1.03,
   'dt_max_penalty': 33.88,
   'tt_max_penalty': 20.97},
  3: {'canon': 198.0,
   'k': 1.18,
   'dt_max_penalty': 27.72,
   'tt_max_penalty': 25.74}})

In [5]:
def get_num_trains(solution):
    scheduled_trains = np.sum(solution.dicrete)
    print("Requested number of trains: ", len(solution.discrete))
    print("Scheduled trains: ", scheduled_trains)
    return scheduled_trains, 0

# 5. Optimize with GSA

In [7]:
sm = RevenueMaximization(schedule, revenue, safe_headway=np.float64(10.0))

In [12]:
def gsa_feasibility(solution):
    S_i = solution.discrete
    print("Test: ", S_i)
    return sm._departure_time_feasibility(S_i)

In [13]:
boundaries = Boundaries(real=[], discrete=[(0, 1) for _ in range(len(schedule))])

gsa_algo = GSA(objective_function=get_num_trains,
               is_feasible=gsa_feasibility,
               r_dim=0,
               d_dim=len(schedule),
               boundaries=boundaries)

In [14]:
gsa_algo.set_seed(seed=28)

training_history = gsa_algo.optimize(population_size=5,
                                     iters=10,
                                     chaotic_constant=False,
                                     repair_solution=False)

Initializing positions of the individuals in the population...
Test:  [1 0 1]
Test:  [1 1 1]
Test:  0


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()